In [2]:
import torch
import torch.nn as nn
import math

In [ ]:
class LoRALinear(nn.Module):
    def __init__(self, in_features, out_features, r=4, lora_alpha=1):
        super().__init__()
        # 1. 原来的全连接层 (模拟冻结的预训练权重)
        self.pretrained = nn.Linear(in_features, out_features, bias=False)#提取预训练的线性层

        # --- 核心：冻结它！不许更新梯度 ---
        self.pretrained.weight.requires_grad = False 

        self.lora_A = nn.Parameter(torch.randn(r, in_features))
        self.lora_B = nn.Parameter(torch.zeros(out_features, r))

        nn.init.kaiming_uniform_(self.lora_A,a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)

        self.scaling=lora_alpha/r#缩放因子，使无论r值变化如何LoRA的初始权重变化幅度一致

    def forward(self,x):
        #最终结果=原始结果+增量
        #1.原始结果@W.T
        result_base=self.pretrained(x)

        #2.增量x@A.T@B.T*scaling
        lora_process=(x@self.lora_A.T)@self.lora_B.T*self.scaling

        return result_base+lora_process

In [5]:
#test code
#假设拥有一个巨大的层输入1024输出1024
layer=LoRALinear(1024,1024,r=8)

total_params=sum(p.numel() for p in layer.parameters())
trainable_params=sum(p.numel() for p in layer.parameters() if p.requires_grad)

print(f"原始权重参数量：{1024*1024}")
print(f"LoRA (A+B) 参数量: {trainable_params}")
print(f"节省了{(1-trainable_params/(1024*1024))*100:.2f}%的显存")

#测试前向传播
input_data=torch.randn(1,1024)#randn是一个造数机器，生成符合标准正态分布（均值为0，方差为1）的随机数。
out_put=layer(input_data)
print("输出维度：",out_put.shape)#证明最后的输出维度应该不会发生改变


原始权重参数量：1048576
LoRA (A+B) 参数量: 16384
节省了98.44%的显存
输出维度： torch.Size([1, 1024])


In [6]:
if torch.cuda.is_available():
    print("yep")

yep
